## spark read s3 json

In [ ]:
! pip install psycopg2-binary

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window, Row, SparkSession

import psycopg2
import pprint
import boto3
import json
import sys
import os

pp = pprint.PrettyPrinter(indent = 3)
print('imported modules.')

# Set Java home environment variable
# os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/temurin-8.jdk/Contents/Home'  # Update this path to match your Java installation

imported modules.


In [11]:
from pathlib import Path
import os

# Get the notebook directory path
notebook_dir = Path(os.getcwd())
creds_path = notebook_dir / "notebook/creds.json"

# read creds.json
try:
    with open(creds_path, "r") as f:
        creds = json.load(f)
        print(f"Credentials loaded from: {creds_path}")
except FileNotFoundError:
    print(f"Error: creds.json not found at {creds_path}")
    print("Please create the credentials file with required AWS and PostgreSQL settings")
    raise

Error: creds.json not found at /opt/workspace/notebook/creds.json
Please create the credentials file with required AWS and PostgreSQL settings


FileNotFoundError: [Errno 2] No such file or directory: '/opt/workspace/notebook/creds.json'

## Spark

In [5]:
# Stop any existing Spark session
if 'spark' in locals():
    spark.stop()

## start spark

In [ ]:
try:
    # Create Spark session with required configurations
    spark = SparkSession.builder \
        .appName("YelpAnalysis") \
        .master("spark://10.0.0.9:7077") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .config("spark.worker.memory", "4g") \
        .config("spark.executor.core", "4") \
        .config("spark.cores.max", 4) \
        .config("spark.hadoop.fs.s3a.access.key", creds["aws_client"]) \
        .config("spark.hadoop.fs.s3a.secret.key", creds["aws_secret"]) \
        .config("spark.jars.packages", 
                "org.apache.hadoop:hadoop-aws:3.3.4," + 
                "org.apache.hadoop:hadoop-common:3.3.4," +
                "org.apache.logging.log4j:log4j-slf4j-impl:2.17.2," +
                "org.apache.logging.log4j:log4j-api:2.17.2," +
                "org.apache.logging.log4j:log4j-core:2.17.2," + 
                "org.apache.hadoop:hadoop-client:3.3.4," + 
                "io.delta:delta-core_2.12:2.4.0," + 
                "org.postgresql:postgresql:9.4.1212") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
    
        # .config("spark.jars.packages", "org.apache.hadoop:hadoop-common:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,org.apache.hadoop:hadoop-client:3.3.4,io.delta:delta-core_2.12:2.4.0,org.postgresql:postgresql:9.4.1212") \
        
    
except Exception as e:
    print(str(e))

name 'creds' is not defined


### list s3 files

In [75]:
client = boto3.client('s3',
                    aws_access_key_id=creds["aws_client"],
                    aws_secret_access_key= creds["aws_secret"])
bucket = "yelp-stevenhurwitt-2"
file = "yelp_academic_dataset_business.json"

bucket_meta = client.list_objects(Bucket = 'yelp-stevenhurwitt-2')
print('files in s3 bucket:')
print('')
for c in bucket_meta['Contents']:
    print(c['Key'])

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


files in s3 bucket:

business/_delta_log/00000000000000000000.json
business/_delta_log/00000000000000000001.json
business/_delta_log/00000000000000000002.json
business/_delta_log/00000000000000000003.json
business/_symlink_format_manifest/state=AB/manifest
business/_symlink_format_manifest/state=AZ/manifest
business/_symlink_format_manifest/state=CA/manifest
business/_symlink_format_manifest/state=CO/manifest
business/_symlink_format_manifest/state=DE/manifest
business/_symlink_format_manifest/state=FL/manifest
business/_symlink_format_manifest/state=HI/manifest
business/_symlink_format_manifest/state=ID/manifest
business/_symlink_format_manifest/state=IL/manifest
business/_symlink_format_manifest/state=IN/manifest
business/_symlink_format_manifest/state=LA/manifest
business/_symlink_format_manifest/state=MA/manifest
business/_symlink_format_manifest/state=MI/manifest
business/_symlink_format_manifest/state=MO/manifest
business/_symlink_format_manifest/state=MT/manifest
business/_symli

## spark read json

### schemas

In [5]:
# Define schemas for each dataset
checkin_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("date", StringType(), True)
])

review_schema = StructType([
    StructField("review_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("business_id", StringType(), True),
    StructField("stars", FloatType(), True),
    StructField("useful", IntegerType(), True),
    StructField("funny", IntegerType(), True),
    StructField("cool", IntegerType(), True),
    StructField("text", StringType(), True),
    StructField("date", TimestampType(), True)
])

tip_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("business_id", StringType(), True),
    StructField("text", StringType(), True),
    StructField("date", TimestampType(), True),
    StructField("compliment_count", IntegerType(), True)
])

user_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("yelping_since", TimestampType(), True),
    StructField("friends", StringType(), True),
    StructField("useful", IntegerType(), True),
    StructField("funny", IntegerType(), True),
    StructField("cool", IntegerType(), True),
    StructField("fans", IntegerType(), True),
    StructField("elite", StringType(), True),
    StructField("average_stars", FloatType(), True),
    StructField("compliment_hot", IntegerType(), True),
    StructField("compliment_more", IntegerType(), True),
    StructField("compliment_profile", IntegerType(), True),
    StructField("compliment_cute", IntegerType(), True),
    StructField("compliment_list", IntegerType(), True),
    StructField("compliment_note", IntegerType(), True),
    StructField("compliment_plain", IntegerType(), True),
    StructField("compliment_cool", IntegerType(), True),
    StructField("compliment_funny", IntegerType(), True),
    StructField("compliment_writer", IntegerType(), True),
    StructField("compliment_photos", IntegerType(), True)
])

business_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("stars", DoubleType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("attributes", MapType(StringType(), StringType()), True),
    StructField("categories", StringType(), True),
    StructField("hours", MapType(StringType(), StringType()), True)
])

## load yelp dataframes

In [6]:
# Function to load dataframes
def load_yelp_dataframes(base_path):
    try:
        # Load each dataset
        df_business = spark.read.schema(business_schema)\
            .json(f"{base_path}/yelp_academic_dataset_business.json")
        
        df_checkin = spark.read.schema(checkin_schema)\
            .json(f"{base_path}/yelp_academic_dataset_checkin.json")
        
        df_review = spark.read.schema(review_schema)\
            .json(f"{base_path}/yelp_academic_dataset_review.json")
        
        df_tip = spark.read.schema(tip_schema)\
            .json(f"{base_path}/yelp_academic_dataset_tip.json")
        
        df_user = spark.read.schema(user_schema)\
            .json(f"{base_path}/yelp_academic_dataset_user.json")
        
        # Print basic info about loaded dataframes
        print(f"Businesses: {df_business.count()} rows")
        print(f"Checkins: {df_checkin.count()} rows")
        print(f"Reviews: {df_review.count()} rows")
        print(f"Tips: {df_tip.count()} rows")
        print(f"Users: {df_user.count()} rows")
        
        return df_business, df_checkin, df_review, df_tip, df_user
    
    except Exception as e:
        print(f"Error loading dataframes: {str(e)}")
        return None

## read data

http://localhost:4040/jobs

In [39]:
# Usage
base_path = "s3a://yelp-stevenhurwitt-2/"  # Update this path
df_business, df_checkin, df_review, df_tip, df_user = load_yelp_dataframes(base_path)


Businesses: 150346 rows


Checkins: 131930 rows


ERROR:root:KeyboardInterrupt while sending command.                (2 + 4) / 40]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

### verify schemas

In [25]:
# Verify schemas
print("\nCheckin Schema:")
df_checkin.printSchema()
print("\nReview Schema:")
df_review.printSchema()
print("\nTip Schema:")
df_tip.printSchema()
print("\nUser Schema:")
df_user.printSchema()


Checkin Schema:
root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)


Review Schema:
root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: float (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- cool: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- date: timestamp (nullable = true)


Tip Schema:
root
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- compliment_count: integer (nullable = true)


User Schema:
root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- yelping_since: timestamp (nullable = true)
 |-- friends: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)


## write to delta on s3

In [12]:
def write_to_s3_delta(df, bucket, folder, partition_cols=None):
    """
    Write DataFrame to S3 in Delta format
    Args:
        df: Spark DataFrame
        bucket: S3 bucket name
        folder: Folder name for the dataset
        partition_cols: List of columns to partition by (optional)
    """
    try:
        writer = df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .option("mergeSchema", "true")
        
        if partition_cols:
            writer = writer.partitionBy(partition_cols)
            
        path = f"s3a://{bucket}/{folder}/"
        writer.save(path)
        print(f"Successfully wrote {folder} data to Delta table at {path}")
        
        # Create Delta table if it doesn't exist
        spark.sql(f"""
        CREATE TABLE IF NOT EXISTS delta.`{path}`
        USING DELTA
        """)
        
    except Exception as e:
        print(f"Error writing {folder} to Delta: {str(e)}")

# Usage example:
bucket = "yelp-stevenhurwitt-2"

try:
    # Reviews - partition by year and month
    df_review = df_review.withColumn(
        "year", year("date")
    ).withColumn(
        "month", month("date")
    )
    write_to_s3_delta(df_review, bucket, "reviews", ["year", "month"])

    # Users - no partitioning
    write_to_s3_delta(df_user, bucket, "users")

    # Tips - partition by year
    df_tip = df_tip.withColumn("year", year("date"))
    write_to_s3_delta(df_tip, bucket, "tips", ["year"])

    # Checkins - no partitioning
    write_to_s3_delta(df_checkin, bucket, "checkins")

    # Business - partition by state
    df_business = df_business.fillna({'state': 'UNKNOWN'})
    write_to_s3_delta(df_business, bucket, "business", ["state"])

    print("All datasets written to Delta format successfully")
    
except Exception as e:
    print(f"Error in write operations: {str(e)}")

Error in write operations: name 'df_review' is not defined


In [33]:
try:
    # Business - partition by state
    # First verify state column exists and handle nulls
    df_business = df_business.fillna({'state': 'UNKNOWN'})
    
    # Write to Delta with state partitioning
    write_to_s3_delta(df_business, bucket, "business", ["state"])
    
    # Verify partitioning
    partitions = spark.sql(f"SHOW PARTITIONS delta.`s3a://{bucket}/business`")
    print("\nPartitions created:")
    partitions.show()

except Exception as e:
    print(f"Error writing business data: {str(e)}")

25/03/12 01:01:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Successfully wrote business data to Delta table at s3a://yelp-stevenhurwitt-2/business/
Error writing business data: Table spark_catalog.delta.`s3a://yelp-stevenhurwitt-2/business` does not support partition management.; line 1 pos 16;
ShowPartitions [partition#1719]
+- ResolvedTable org.apache.spark.sql.delta.catalog.DeltaCatalog@535e64b8, delta.`s3a://yelp-stevenhurwitt-2/business`, DeltaTableV2(org.apache.spark.sql.SparkSession@5e2bd3b1,s3a://yelp-stevenhurwitt-2/business,None,None,None,Map(),org.apache.spark.sql.util.CaseInsensitiveStringMap@1f), [business_id#1720, name#1721, address#1722, city#1723, state#1724, postal_code#1725, latitude#1726, longitude#1727, stars#1728, review_count#1729, is_open#1730, attributes#1731, categories#1732, hours#1733]



In [35]:
# Verify data access by partition
df_check = spark.read.format("delta").load(f"s3a://{bucket}/business")
df_check.groupBy("state").count().orderBy("count", ascending=False).show()

+-----+-----+
|state|count|
+-----+-----+
|   PA|34039|
|   FL|26330|
|   TN|12056|
|   IN|11247|
|   MO|10913|
|   LA| 9924|
|   AZ| 9912|
|   NJ| 8536|
|   NV| 7715|
|   AB| 5573|
|   CA| 5203|
|   ID| 4467|
|   DE| 2265|
|   IL| 2145|
|   TX|    4|
|   CO|    3|
|   WA|    2|
|   MA|    2|
|   HI|    2|
|   MI|    1|
+-----+-----+
only showing top 20 rows



## read delta

In [7]:
df_business = spark.read.format("delta").load("s3a://yelp-stevenhurwitt-2/business")
print(f"business rows: {df_business.count()}")

25/03/12 01:56:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/03/12 01:57:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


business rows: 150346


In [8]:
df_checkins = spark.read.format("delta").load("s3a://yelp-stevenhurwitt-2/checkins")
print(f"checkins rows: {df_checkins.count()}")

checkins rows: 131930


In [9]:
df_reviews = spark.read.format("delta").load("s3a://yelp-stevenhurwitt-2/reviews")
print(f"reviews rows: {df_reviews.count()}")

reviews rows: 6990280


In [10]:
df_tips = spark.read.format("delta").load("s3a://yelp-stevenhurwitt-2/tips")
print(f"tips rows: {df_tips.count()}")

tips rows: 908915


In [11]:
df_users = spark.read.format("delta").load("s3a://yelp-stevenhurwitt-2/users")
print(f"users rows: {df_users.count()}")

users rows: 1987897


# df reviews

In [56]:
print(f"rows: {df_reviews.count()}.")
df_reviews.show()

rows: 6990280.


+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|year|month|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+----+-----+
|O7Y8uu3rkVB3ogt1I...|kQX4IZc95PkbkMcS9...|XnQ84ylyAZwh-XfHG...|  4.0|     2|    0|   1|This place has ev...|2005-07-26 17:28:02|2005|    7|
|qRuLSc3dAIJHtbkDD...|98pdN0Hn3Vx_qBlv2...|qclZoDz3sjT7v5xOS...|  3.0|     2|    0|   0|If you must ventu...|2005-07-11 07:05:31|2005|    7|
|3dcG_3BA6-eruZ09t...|n-lBS02-3yvlY5Q91...|4cZsFnpFPWvPet0_X...|  4.0|     1|    0|   0|Cozy neighborhood...|2005-07-11 02:03:13|2005|    7|
|SrEmDb9BHIlOsznaM...|qG_bVdazAjjAO1l94...|103cmgKE0n1Cb8QCE...|  5.0|     1|    0|   0|Home to the only ...|2005-07-10 22:47:38|2005|    7|
|Mg79Y_WJe8EF

In [ ]:
df_users.orderBy("review_count").show()

In [58]:
df_business.show()

+--------------------+--------------------+--------------------+--------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|         business_id|                name|             address|          city|state|postal_code|     latitude|     longitude|stars|review_count|is_open|          attributes|          categories|               hours|
+--------------------+--------------------+--------------------+--------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|CSOoD6D7NWvg--R8P...|       Sugared Salon|                    |    Clearwater|   FL|      33755|    27.981827|   -82.7886395|  5.0|           6|      1|{ByAppointmentOnl...|Spray Tanning, Ta...|{Monday -> 0:0-0:...|
|hHX17fJwcXnEABeJl...|AAMCO Transmissio...|   27989 US Hwy 19 N|    Clearwater|   FL|      33761|28.0266915563|-82.7374593669|  4.5|

# write to postgres

## create tables

In [ ]:
def create_postgres_tables():
    """Create PostgreSQL tables in correct dependency order"""
    try:
        conn = psycopg2.connect(
            host=creds["postgres_host"],
            database=creds["postgres_db"],
            user=creds["postgres_user"],
            password=creds["postgres_password"]
        )
        
        with conn.cursor() as cur:
            # Read SQL statements
            with open('create_tables.sql', 'r') as f:
                sql = f.read()
                
            # Execute statements
            cur.execute(sql)
            conn.commit()
            print("Tables created successfully")
            
    except Exception as e:
        print(f"Error creating tables: {str(e)}")
        conn.rollback()
    finally:
        conn.close()

# Execute table creation
create_postgres_tables()

## write to postgres from spark

In [ ]:
def write_to_postgres(df, table_name):
    """
    Write Spark DataFrame to PostgreSQL
    
    Args:
        df: Spark DataFrame
        table_name: Name of the target PostgreSQL table
    """
    try:
        # PostgreSQL connection properties
        postgres_props = {
            "url": f"jdbc:postgresql://{creds['postgres_host']}:5433/{creds['postgres_db']}",
            "driver": "org.postgresql.Driver",
            "user": creds["postgres_user"],
            "password": creds["postgres_password"]
        }
        
        # Write DataFrame to PostgreSQL
        df.write \
            .format("jdbc") \
            .option("url", postgres_props["url"]) \
            .option("driver", postgres_props["driver"]) \
            .option("dbtable", table_name) \
            .option("user", postgres_props["user"]) \
            .option("password", postgres_props["password"]) \
            .mode("overwrite") \
            .save()
            
        print(f"Successfully wrote {table_name} to PostgreSQL")
        
    except Exception as e:
        print(f"Error writing {table_name} to PostgreSQL: {str(e)}")

# Write all datasets to PostgreSQL
try:
    # Add PostgreSQL JDBC driver to Spark
    # spark.sparkContext.addJar("postgresql-42.2.23.jar")
    
    # Write each dataset
    write_to_postgres(df_business, "business")
    write_to_postgres(df_reviews, "review")
    write_to_postgres(df_users, "user_profile")  # using user_profile since user is reserved
    write_to_postgres(df_tips, "tip")
    write_to_postgres(df_checkins, "checkin")
    
    print("All datasets written to PostgreSQL successfully")
    
except Exception as e:
    print(f"Error in PostgreSQL write operations: {str(e)}")

Error writing business to PostgreSQL: An error occurred while calling o69.save.
: org.postgresql.util.PSQLException: The authentication type 10 is not supported. Check that you have configured the pg_hba.conf file to include the client's IP address or subnet, and that it is using an authentication scheme supported by the driver.
	at org.postgresql.core.v3.ConnectionFactoryImpl.doAuthentication(ConnectionFactoryImpl.java:634)
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:217)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:52)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:216)
	at org.postgresql.Driver.makeConnection(Driver.java:404)
	at org.postgresql.Driver.connect(Driver.java:272)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionPro

# aws athena

### create glue database

In [13]:
def create_glue_database(database_name):
    """
    Create AWS Glue Database if it doesn't exist
    """
    try:
        glue = boto3.client('glue',
                           region_name=creds["aws_region"],
                           aws_access_key_id=creds['aws_client'],
                           aws_secret_access_key=creds['aws_secret'])
        
        # Check if database exists
        try:
            glue.get_database(Name=database_name)
            print(f"Database {database_name} already exists")
        except glue.exceptions.EntityNotFoundException:
            # Create database if it doesn't exist
            glue.create_database(
                DatabaseInput={
                    'Name': database_name,
                    'Description': 'Database for Yelp Analytics'
                }
            )
            print(f"Created database {database_name}")
            
    except Exception as e:
        print(f"Error creating database: {str(e)}")

# Add this before your existing table creation code
bucket = "yelp-stevenhurwitt-2"
database_name = "yelp_analytics"

# Create database first
create_glue_database(database_name)

Database yelp_analytics already exists


### write to s3 delta w/ manifest

In [14]:
def write_to_s3_delta_with_manifest(df, bucket, folder, partition_cols=None):
    """
    Write DataFrame to S3 in Delta format and generate manifest for Athena
    """
    try:
        # Write Delta table
        path = f"s3a://{bucket}/{folder}/"
        
        writer = df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .option("mergeSchema", "true")
        
        if partition_cols:
            writer = writer.partitionBy(partition_cols)
            
        writer.save(path)
        
        # Generate manifest for Athena
        spark.sql(f"""
        GENERATE symlink_format_manifest FOR TABLE delta.`{path}`
        """)
        
        print(f"Successfully wrote {folder} data and manifest to {path}")
        
    except Exception as e:
        print(f"Error writing {folder}: {str(e)}")

# AWS Glue catalog registration function
def register_with_glue(database_name, table_name, s3_path, partition_cols=None):
    """
    Register Delta table with AWS Glue catalog with duplicate column check
    """
    try:
        # Get table schema from Spark
        schema = spark.read.format("delta").load(s3_path).schema
        
        # Convert Spark schema to Glue columns with duplicate check
        glue_columns = []
        seen_columns = set()
        
        for field in schema.fields:
            # Skip if column name already exists or is in partition cols
            if field.name in seen_columns or (partition_cols and field.name in partition_cols):
                print(f"Warning: Skipping duplicate column {field.name}")
                continue
                
            seen_columns.add(field.name)
            glue_columns.append({
                'Name': field.name,
                'Type': field.dataType.simpleString()
            })
        
        # Create Glue client
        glue = boto3.client('glue',
                           region_name=creds["aws_region"],
                           aws_access_key_id=creds['aws_client'],
                           aws_secret_access_key=creds['aws_secret'])
        
        # Delete existing table if it exists
        try:
            glue.delete_table(DatabaseName=database_name, Name=table_name)
            print(f"Deleted existing table {table_name}")
        except glue.exceptions.EntityNotFoundException:
            pass
        
        # Create table definition
        table_input = {
            'Name': table_name,
            'StorageDescriptor': {
                'Columns': glue_columns,
                'Location': s3_path,
                'InputFormat': 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat',
                'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
                'SerdeInfo': {
                    'SerializationLibrary': 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
                }
            },
            'TableType': 'EXTERNAL_TABLE'
        }
        
        if partition_cols:
            table_input['PartitionKeys'] = [{'Name': col, 'Type': 'string'} for col in partition_cols]
        
        # Create table in Glue
        glue.create_table(
            DatabaseName=database_name,
            TableInput=table_input
        )
        
        print(f"Successfully registered table {table_name} in Glue catalog")
        
    except Exception as e:
        print(f"Error registering with Glue: {str(e)}")

# Usage example:
bucket = "yelp-stevenhurwitt-2"
database_name = "yelp_analytics"

In [ ]:
try:
    # Business - partition by state
    df_business = df_business.fillna({'state': 'UNKNOWN'})
    write_to_s3_delta_with_manifest(df_business, bucket, "business", ["state"])
    register_with_glue(database_name, "business", f"s3a://{bucket}/business", ["state"])
    
    # Reviews - partition by year and month
    df_reviews = df_reviews.withColumn("year", year("date")).withColumn("month", month("date"))
    write_to_s3_delta_with_manifest(df_reviews, bucket, "reviews", ["year", "month"])
    register_with_glue(database_name, "reviews", f"s3a://{bucket}/reviews", ["year", "month"])

    # Users
    write_to_s3_delta_with_manifest(df_users, bucket, "users")
    register_with_glue(database_name, "users", f"s3a://{bucket}/users")

    # Tips - partition by year
    df_tips = df_tips.withColumn("year", year("date"))
    write_to_s3_delta_with_manifest(df_tips, bucket, "tips", ["year"])
    register_with_glue(database_name, "tips", f"s3a://{bucket}/tips", ["year"])

    # Checkins
    write_to_s3_delta_with_manifest(df_checkins, bucket, "checkins")
    register_with_glue(database_name, "checkins", f"s3a://{bucket}/checkins")

    print("All tables written and registered successfully")
    
except Exception as e:
    print(f"Error in operations: {str(e)}")

Successfully wrote business data and manifest to s3a://yelp-stevenhurwitt-2/business/
Deleted existing table business
Successfully registered table business in Glue catalog


In [ ]:
# Business - partition by state
df_business = df_business.fillna({'state': 'UNKNOWN'})
write_to_s3_delta_with_manifest(df_business, bucket, "business", ["state"])
register_with_glue(database_name, "business", f"s3a://{bucket}/business", ["state"])

In [ ]:
# Reviews - partition by year and month
df_reviews = df_reviews.withColumn("year", year("date")).withColumn("month", month("date"))
write_to_s3_delta_with_manifest(df_reviews, bucket, "reviews", ["year", "month"])
register_with_glue(database_name, "reviews", f"s3a://{bucket}/reviews", ["year", "month"])

ERROR:root:KeyboardInterrupt while sending command.][Stage 69:>  (0 + 0) / 50]
Traceback (most recent call last):
  File "/home/steven/Documents/yelp/.venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/steven/Documents/yelp/.venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Users
write_to_s3_delta_with_manifest(df_users, bucket, "users")
register_with_glue(database_name, "users", f"s3a://{bucket}/users")

In [ ]:
# Tips - partition by year
df_tips = df_tips.withColumn("year", year("date"))
write_to_s3_delta_with_manifest(df_tips, bucket, "tips", ["year"])
register_with_glue(database_name, "tips", f"s3a://{bucket}/tips", ["year"])


In [ ]:
 # Checkins
write_to_s3_delta_with_manifest(df_checkins, bucket, "checkins")
register_with_glue(database_name, "checkins", f"s3a://{bucket}/checkins")

## read from athena

In [ ]:
def read_from_athena(table_name):
    """
    Read table from Athena using Spark
    """
    try:
        # Read from Athena
        df = spark.read \
            .format("delta") \
            .option("cloudFiles.region", creds["aws_region"]) \
            .load(f"s3a://{bucket}/{table_name}")
        
        return df
    
    except Exception as e:
        print(f"Error reading {table_name} from Athena: {str(e)}")
        return None

# Read all tables
try:
    # Read business data
    df_business = read_from_athena("business")
    print("Business records:", df_business.count())
    
    # Read reviews data
    df_reviews = read_from_athena("reviews")
    print("Review records:", df_reviews.count())
    
    # Read users data
    df_users = read_from_athena("users")
    print("User records:", df_users.count())
    
    # Read tips data
    df_tips = read_from_athena("tips")
    print("Tip records:", df_tips.count())
    
    # Read checkins data
    df_checkins = read_from_athena("checkins")
    print("Checkin records:", df_checkins.count())
    
    # Show sample data from each table
    print("\nBusiness Sample:")
    df_business.show(5)
    
    print("\nReviews Sample:")
    df_reviews.show(5)
    
    print("\nUsers Sample:")
    df_users.show(5)
    
    print("\nTips Sample:")
    df_tips.show(5)
    
    print("\nCheckins Sample:")
    df_checkins.show(5)

except Exception as e:
    print(f"Error reading from Athena: {str(e)}")

## read json

In [3]:
def read_json(filename):
    """
    reads a yelp .json file from s3 bucket.

    keyword arguments:
    filename - name of file (str)

    returns: json_file (json)
    """

    bucket = "yelp-dataset-stevenhurwitt"
    file = "raw/yelp_academic_dataset_business.json"
    response = client.get_object(Bucket = bucket, Key = file)
    
    file_content = response['Body'].read().decode('utf-8')
    json_file = json.loads("[" + file_content.replace("}\n{", "},\n{") + "]")
    return(json_file)

## read json files

In [4]:
business_file = read_json("raw/yelp_academic_dataset_business.json")
checkin_file = read_json("raw/yelp_academic_dataset_checkin.json")
review_file = read_json("raw/yelp_academic_dataset_review.json")
tip_file = read_json("raw/yelp_academic_dataset_tip.json")
user_file = read_json("raw/yelp_academic_dataset_user.json")

print("read json files from s3.")

read json files from s3.


### review

In [29]:
pp.pprint(review_file[0])

# {  'address': '1616 Chapala St, Ste 2',
#    'attributes': {'ByAppointmentOnly': 'True'},
#    'business_id': 'Pns2l4eNsfO8kk83dixA6A',
#    'categories': 'Doctors, Traditional Chinese Medicine, '
#                  'Naturopathic/Holistic, Acupuncture, Health & Medical, '
#                  'Nutritionists',
#    'city': 'Santa Barbara',
#    'hours': None,
#    'is_open': 0,
#    'latitude': 34.4266787,
#    'longitude': -119.7111968,
#    'name': 'Abby Rappoport, LAC, CMQ',
#    'postal_code': '93101',
#    'review_count': 7,
#    'stars': 5.0,
#    'state': 'CA'}

NameError: name 'review_file' is not defined

### tip

In [6]:
pp.pprint(tip_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### user

In [7]:
pp.pprint(user_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### checkin

In [8]:
pp.pprint(checkin_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### business

In [9]:
pp.pprint(business_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


In [10]:
dynamodb = boto3.resource('dynamodb', endpoint_url="https://us-east-2.console.aws.amazon.com?arn=arn:aws:dynamodb:us-east-2134132211607:8000")
print('created dynamo resource.')

# try:
#     yelp_business = dynamodb.create_table(
#             TableName='yelp.business',
#             KeySchema=[
#                 {
#                     'AttributeName': 'business_id',
#                     'KeyType': 'HASH'  # Partition key
#                 }
#             ],
#             AttributeDefinitions=[
#                 {
#                     'AttributeName': 'name',
#                     'AttributeType': 'S'
#                 }
#             ],
#             ProvisionedThroughput={
#                 'ReadCapacityUnits': 25,
#                 'WriteCapacityUnits': 20
#             }
#         )
#     print('created dynamo table.')

# except Exception as e:
#     print(f"exception: {e}")
#     print("failed to create dynamo table.")

created dynamo resource.


In [11]:
print(bucket) 

yelp-dataset-stevenhurwitt


In [12]:
print(file)

raw/yelp_academic_dataset_business.json


In [13]:
print(len(business_file))
print(len(checkin_file))
# review_file = read_json("raw/yelp_academic_dataset_review.json")
# tip_file = read_json("raw/yelp_academic_dataset_tip.json")
# user_file = read_json("raw/yelp_academic_dataset_user.json")


150346
150346


In [14]:
print('business json file has {} records with size of {} mb.'.format(len(business_file), sys.getsizeof(business_file)/1000000))
print('tip json file has {} records with size of {} mb.'.format(len(tip_file), sys.getsizeof(tip_file)/1000000))
print('user json file has {} records with size of {} mb.'.format(len(user_file), sys.getsizeof(user_file)/1000000))
print('review json file has {} records with size of {} mb.'.format(len(review_file), sys.getsizeof(review_file)/1000000))
print('checkin json file has {} records with size of {} mb.'.format(len(checkin_file), sys.getsizeof(checkin_file)/1000000))


business json file has 150346 records with size of 1.28316 mb.
tip json file has 150346 records with size of 1.28316 mb.
user json file has 150346 records with size of 1.28316 mb.
review json file has 150346 records with size of 1.28316 mb.
checkin json file has 150346 records with size of 1.28316 mb.


In [15]:
# df_pandas = business.toPandas()

In [16]:
# html_df = df_pandas.to_html()

In [17]:
# display(html_df)

In [18]:
# ! gcloud auth login --no-launch-browser